In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, RocCurveDisplay
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [3]:
credit = pd.read_csv('../data/train_cleaned.csv')
credit.head()

,ID,is_delinquent,length_of_credit,number_of_delinquent_months,average_delinquency_rate,3mo_delinquency,6mo_delinquency,12mo_delinquency,FLAG_OWN_CAR,FLAG_OWN_REALTY,...,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE,YEARS_EMPLOYED
0,5008804,0,16,2,0.125000,0,0,0,Y,Y,...,Civil marriage,Rented apartment,1,1,0,0,missing,2.0,32,12
1,5008805,0,15,2,0.133333,0,0,0,Y,Y,...,Civil marriage,Rented apartment,1,1,0,0,missing,2.0,32,12
2,5008806,0,30,7,0.233333,0,0,1,Y,Y,...,Married,House / apartment,1,0,0,0,Security staff,2.0,58,3
3,5008808,1,5,2,0.400000,1,1,1,N,Y,...,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,52,8
4,5008809,0,5,0,0.000000,0,0,0,N,Y,...,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,52,8


In [4]:
credit.columns

Index(['ID', 'is_delinquent', 'length_of_credit',
       'number_of_delinquent_months', 'average_delinquency_rate',
       '3mo_delinquency', '6mo_delinquency', '12mo_delinquency',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'AGE',
       'YEARS_EMPLOYED'],
      dtype='object')

In [5]:
credit.columns = credit.columns.str.lower()
credit.columns

Index(['id', 'is_delinquent', 'length_of_credit',
       'number_of_delinquent_months', 'average_delinquency_rate',
       '3mo_delinquency', '6mo_delinquency', '12mo_delinquency',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'name_income_type', 'name_education_type', 'name_family_status',
       'name_housing_type', 'flag_mobil', 'flag_work_phone', 'flag_phone',
       'flag_email', 'occupation_type', 'cnt_fam_members', 'age',
       'years_employed'],
      dtype='object')

In [6]:
X = credit.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '3mo_delinquency', '6mo_delinquency', '12mo_delinquency'])

y = credit['is_delinquent']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train.dtypes

length_of_credit         int64
flag_own_car            object
flag_own_realty         object
cnt_children             int64
amt_income_total       float64
name_income_type        object
name_education_type     object
name_family_status      object
name_housing_type       object
flag_mobil               int64
flag_work_phone          int64
flag_phone               int64
flag_email               int64
occupation_type         object
cnt_fam_members        float64
age                      int64
years_employed           int64
dtype: object

In [9]:
cat_cols = X_train.select_dtypes(include='object').columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

## Train, test, split and column transformer with 3 different targets

In [10]:
X = credit.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '3mo_delinquency', '6mo_delinquency', '12mo_delinquency'])

y_1 = credit['is_delinquent']
y_2 = credit['3mo_delinquency']
y_3 = credit['6mo_delinquency']

X_train, X_val, y_train_1, y_val_1 = train_test_split(X, y_1, test_size=0.2, random_state=42)
X_train, X_val, y_train_2, y_val_2 = train_test_split(X, y_2, test_size=0.2, random_state=42)
X_train, X_val, y_train_3, y_val_3 = train_test_split(X, y_3, test_size=0.2, random_state=42)

cat_cols = X_train.select_dtypes(include='object').columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

sc = StandardScaler()

oh = OneHotEncoder(
    categories='auto', 
    drop='first', 
    sparse_output=False, 
    dtype='int',
    handle_unknown = 'ignore'
)

ct = ColumnTransformer(
    transformers=[
        ('oh', oh, cat_cols),
        ('sc', sc, num_cols)
    ]
)

X_train_ct = ct.fit_transform(X_train)
X_val_ct = ct.transform(X_val)

## Gradient Boosting with default parameters
Predicting the classes with 3 different targets

In [48]:
credit['is_delinquent'].value_counts(normalize=True)

0    0.757308
1    0.242692
Name: is_delinquent, dtype: float64

In [27]:
gb = GradientBoostingClassifier()
gb.fit(X_train_ct, y_train_1)

print(gb.score(X_train_ct, y_train_1))
print(gb.score(X_val_ct, y_val_1))

0.778815196394076
0.7633612363168062


In [49]:
credit['3mo_delinquency'].value_counts(normalize=True)

0    0.589633
1    0.410367
Name: 3mo_delinquency, dtype: float64

In [46]:
gb = GradientBoostingClassifier()
gb.fit(X_train_ct, y_train_2)

print(gb.score(X_train_ct, y_train_2))
print(gb.score(X_val_ct, y_val_2))

0.758813586606568
0.7546683837733419


In [50]:
credit['6mo_delinquency'].value_counts(normalize=True)

0    0.526401
1    0.473599
Name: 6mo_delinquency, dtype: float64

In [52]:
gb = GradientBoostingClassifier()
gb.fit(X_train_ct, y_train_3)

print(gb.score(X_train_ct, y_train_3))
print(gb.score(X_val_ct, y_val_3))

0.7706455247907277
0.766580811332904


## Gradient Boosting with GridSearchCV

In [52]:
pgrids = {
    'learning_rate': [.15, .175, 0.2],
    'n_estimators': [285, 290, 295],
    'max_depth': [8, 9, 10]
}

In [53]:
%%time
gb = GradientBoostingClassifier()
gs = GridSearchCV(gb, param_grid=pgrids, cv=5)
gs.fit(X_train_ct, y_train)

CPU times: user 5min 26s, sys: 1.68 s, total: 5min 28s
Wall time: 5min 33s


GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=0.175,
                                                  max_depth=9),
             param_grid={'n_estimators': [285, 290, 295, 300]})

In [54]:
print(gs.score(X_train_ct, y_train))
print(gs.score(X_val_ct, y_val))

0.9760141661300709
0.7908886027044431


In [102]:
gs.best_params_

{'n_estimators': 290}

## Ada Boost with GridSearchCV

In [ ]:
%%time
tree = DecisionTreeClassifier(random_state=123)#, max_depth=1
ada = AdaBoostClassifier(estimator=tree, random_state=42)

pgrids_ada = {
    'learning_rate': [2.25, 2.5, 2.7],
    'n_estimators': [250, 300, 350],
    'estimator__max_depth': [71, 81, 91, 101, None],
    # 'estimator__min_samples_leaf': [7, 8, 9],
    'estimator__max_features': ['auto', 'sqrt', 'log2']
}


gs_ada = GridSearchCV(ada, param_grid=pgrids_ada, cv=5)
gs_ada.fit(X_train_ct, y_train)

print(gs_ada.score(X_train_ct, y_train))
print(gs_ada.score(X_val_ct, y_val))

In [ ]:
gs_ada.best_params_

In [ ]:
%%time
tree = DecisionTreeClassifier(random_state=123)#, max_depth=1
ada = AdaBoostClassifier(estimator=tree, random_state=42)

pgrids_ada = {
    'learning_rate': [2.25, 2.5, 2.7],
    'n_estimators': [250, 300, 350],
    'estimator__max_depth': [71, 81, 91, 101, None],
    # 'estimator__min_samples_leaf': [7, 8, 9],
    'estimator__max_features': ['auto', 'sqrt', 'log2']
}


gs_ada_2 = GridSearchCV(ada, param_grid=pgrids_ada, cv=5)
gs_ada_2.fit(X_train_ct, y_train_2)

print(gs_ada.score(X_train_ct, y_train_2))
print(gs_ada.score(X_val_ct, y_val_2))

## SCV

In [41]:
credit['is_delinquent'].value_counts(normalize=True)

0    0.757308
1    0.242692
Name: is_delinquent, dtype: float64

In [42]:
svc_1 = SVC(C=1, kernel='rbf', degree = 1)
svc_1.fit(X_train_ct, y_train_1)
# Evaluate model.
svc_1.score(X_val_ct, y_val_1)

0.7548293625241468

In [43]:
credit['3mo_delinquency'].value_counts(normalize=True)

0    0.589633
1    0.410367
Name: 3mo_delinquency, dtype: float64

In [44]:
svc_2 = SVC(C=1, kernel='rbf', degree = 1)
svc_2.fit(X_train_ct, y_train_2)
# Evaluate model.
svc_2.score(X_val_ct, y_val_2)

0.7598197037990985

In [53]:
credit['6mo_delinquency'].value_counts(normalize=True)

0    0.526401
1    0.473599
Name: 6mo_delinquency, dtype: float64

In [54]:
svc_3 = SVC(C=1, kernel='rbf', degree = 1)
svc_3.fit(X_train_ct, y_train_3)
# Evaluate model.
svc_3.score(X_val_ct, y_val_3)

0.7670637475853187

## Dense Neural Network

In [65]:
def dnn(X_train, y_train, X_val, y_val):

    model = Sequential()
    model.add(Dense(256, input_dim=X_train_ct.shape[1], activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy', 'Recall'])

    es = EarlyStopping(monitor='val_loss', patience=5)

    model.fit(
    X_train_ct, 
    y_train_1, 
    epochs=50, 
    batch_size=32, 
    validation_data=(X_val, y_val),
    callbacks = es
    )

    return model

In [69]:
# Model did worse with dropouts, l2 regularizer
# Performed similar with extra hidden layers

In [66]:
is_delinq = dnn(X_train_ct, y_train_1, X_val_ct, y_val_1)

Epoch 1/50
777/777 [==============================] - 1s 1000us/step - loss: 0.4985 - accuracy: 0.7574 - recall: 0.0775 - val_loss: 0.4975 - val_accuracy: 0.7558 - val_recall: 0.0805
Epoch 2/50
777/777 [==============================] - 1s 906us/step - loss: 0.4853 - accuracy: 0.7640 - recall: 0.1883 - val_loss: 0.5098 - val_accuracy: 0.7585 - val_recall: 0.1823
Epoch 3/50
777/777 [==============================] - 1s 907us/step - loss: 0.4798 - accuracy: 0.7692 - recall: 0.2460 - val_loss: 0.4950 - val_accuracy: 0.7556 - val_recall: 0.0993
Epoch 4/50
777/777 [==============================] - 1s 930us/step - loss: 0.4752 - accuracy: 0.7709 - recall: 0.2611 - val_loss: 0.4919 - val_accuracy: 0.7618 - val_recall: 0.1720
Epoch 5/50
777/777 [==============================] - 1s 916us/step - loss: 0.4694 - accuracy: 0.7741 - recall: 0.2885 - val_loss: 0.4934 - val_accuracy: 0.7585 - val_recall: 0.2070
Epoch 6/50
777/777 [==============================] - 1s 917us/step - loss: 0.4620 - accu

In [67]:
three_month_delinq = dnn(X_train_ct, y_train_2, X_val_ct, y_val_2)

Epoch 1/50
777/777 [==============================] - 1s 990us/step - loss: 0.4972 - accuracy: 0.7607 - recall: 0.1297 - val_loss: 0.6444 - val_accuracy: 0.6117 - val_recall: 0.0903
Epoch 2/50
777/777 [==============================] - 1s 960us/step - loss: 0.4856 - accuracy: 0.7642 - recall: 0.1889 - val_loss: 0.6414 - val_accuracy: 0.6077 - val_recall: 0.0807
Epoch 3/50
777/777 [==============================] - 1s 934us/step - loss: 0.4808 - accuracy: 0.7679 - recall: 0.2199 - val_loss: 0.6135 - val_accuracy: 0.6627 - val_recall: 0.2540
Epoch 4/50
777/777 [==============================] - 1s 917us/step - loss: 0.4764 - accuracy: 0.7729 - recall: 0.2480 - val_loss: 0.5959 - val_accuracy: 0.6722 - val_recall: 0.2860
Epoch 5/50
777/777 [==============================] - 1s 915us/step - loss: 0.4710 - accuracy: 0.7773 - recall: 0.2640 - val_loss: 0.6254 - val_accuracy: 0.6425 - val_recall: 0.1864
Epoch 6/50
777/777 [==============================] - 1s 919us/step - loss: 0.4656 - accur

In [68]:
six_month_delinq = dnn(X_train_ct, y_train_3, X_val_ct, y_val_3)

Epoch 1/50
777/777 [==============================] - 1s 1ms/step - loss: 0.4969 - accuracy: 0.7581 - recall: 0.0605 - val_loss: 0.6547 - val_accuracy: 0.5761 - val_recall: 0.1385
Epoch 2/50
777/777 [==============================] - 1s 928us/step - loss: 0.4852 - accuracy: 0.7616 - recall: 0.1779 - val_loss: 0.6458 - val_accuracy: 0.6219 - val_recall: 0.2568
Epoch 3/50
777/777 [==============================] - 1s 922us/step - loss: 0.4803 - accuracy: 0.7689 - recall: 0.2386 - val_loss: 0.6991 - val_accuracy: 0.5435 - val_recall: 0.0530
Epoch 4/50
777/777 [==============================] - 1s 916us/step - loss: 0.4737 - accuracy: 0.7747 - recall: 0.2628 - val_loss: 0.6639 - val_accuracy: 0.6592 - val_recall: 0.3554
Epoch 5/50
777/777 [==============================] - 1s 920us/step - loss: 0.4687 - accuracy: 0.7765 - recall: 0.2975 - val_loss: 0.6843 - val_accuracy: 0.6024 - val_recall: 0.2061
Epoch 6/50
777/777 [==============================] - 1s 931us/step - loss: 0.4615 - accurac